In [1]:
from selenium.webdriver import Safari
from selenium.webdriver.common.keys import Keys

In [16]:
driver = Safari()

In [17]:
driver.get("https://semantle.com/")

In [22]:
driver.execute_script("document.getElementById('rules-underlay').remove()")

In [23]:
elem = driver.execute_script("return document.getElementById('guess')")
elem

<selenium.webdriver.remote.webelement.WebElement (session="B941F5F8-E311-4C97-AC11-E707165D80A3", element="node-F9D8E610-B802-482A-A174-DED2D78711CE")>

In [24]:
elem.send_keys("hello")

In [25]:
elem.send_keys(Keys.ENTER)

In [29]:
e = driver.execute_script("return document.getElementById('guesses').getElementsByTagName('tr')[1].getElementsByTagName('td')[2].innerHTML")

In [126]:
from gensim import models

w = models.KeyedVectors.load_word2vec_format(
    'GoogleNews-vectors-negative300.bin', binary=True)

In [4]:
w

In [7]:
w.similar_by_word("computer")

[('computers', 0.7979379892349243),
 ('laptop', 0.6640493273735046),
 ('laptop_computer', 0.6548868417739868),
 ('Computer', 0.6473334431648254),
 ('com_puter', 0.6082079410552979),
 ('technician_Leonard_Luchko', 0.5662748217582703),
 ('mainframes_minicomputers', 0.5617721080780029),
 ('laptop_computers', 0.5585449934005737),
 ('PC', 0.5539618134498596),
 ('maker_Dell_DELL.O', 0.5519254207611084)]

In [11]:
len(w.index2word)

3000000

In [14]:
filtered_word2index = dict((w.index2word[i], i) for i in range(len(w.index2word)) if "_" not in w.index2word[i])

In [15]:
len(filtered_word2index)

929022

In [17]:
import string

In [20]:
filtered_word2index_2 = dict((w.index2word[i], i) for i in filtered_word2index.values() if not any(j.isdigit() or j in string.punctuation for j in w.index2word[i]))

In [21]:
len(filtered_word2index_2)

869757

In [23]:
list(filtered_word2index_2.values())[:10]

[1, 2, 3, 4, 5, 7, 8, 9, 10, 11]

In [32]:
import json

In [50]:
with open("filtered_word2index.json", "w") as filt_w2i_json:
	json.dump([i.lower() for i in filtered_word2index_2.keys()], filt_w2i_json)

In [42]:
import numpy as np

In [43]:
arr_indices = np.array(list(filtered_word2index_2.values()))

In [44]:
arr_indices.shape

(869757,)

In [45]:
arr_indices[:5]

array([1, 2, 3, 4, 5])

In [46]:
new_np_vec = w.vectors[arr_indices]

In [47]:
new_np_vec.shape

(869757, 300)

In [48]:
with open("embed_vec_filtered.npy", "wb") as filtered_npy:
	np.save(filtered_npy, new_np_vec)

In [103]:
def normalize(v):
	norm=np.linalg.norm(v, axis = -1)
	norm += np.float32(norm == 0.)
	return norm

In [107]:
def cosine_similarity(x, y):
	x_norm = normalize(x)
	y_norm = normalize(y)

	return np.sum(x * y, axis = -1) / (x_norm * y_norm)

In [108]:
import numpy as np

In [109]:
vec_a = np.random.random((10, 15))

In [110]:
vec_b = np.random.random((1,15))

In [111]:
cosine_similarity(vec_a, vec_b).shape

(10,)

In [83]:
[i.lower() for i in filtered_word2index_2.keys()][124265]

'hails'

In [84]:
a = np.array([i for i in range(len(filtered_word2index_2))])

In [85]:
b = np.float32(a == 124265) * 1000.

In [86]:
b[:5]

array([0., 0., 0., 0., 0.], dtype=float32)

In [87]:
import tensorflow as tf

In [88]:
v = np.random.random((10, 10))

In [96]:
v_a = np.linalg.norm(v, axis = -1)

In [97]:
v_a.shape

(10,)

In [91]:
v_b = tf.norm(v, axis = -1)
tf.shape(v_b)

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([10], dtype=int32)>

In [93]:
v_b.numpy()

array([1.87223984, 1.62621227, 2.17194851, 1.836867  , 2.02918624,
       1.58326231, 1.87052073, 1.88296791, 1.65865777, 1.02088947])

In [98]:
v_a

array([1.87223984, 1.62621227, 2.17194851, 1.836867  , 2.02918624,
       1.58326231, 1.87052073, 1.88296791, 1.65865777, 1.02088947])

In [99]:
v_a * v_a

array([3.50528201, 2.64456634, 4.71736035, 3.37408038, 4.11759678,
       2.50671955, 3.4988478 , 3.54556817, 2.7511456 , 1.04221531])

In [102]:
v.shape

(10, 10)

In [112]:
vec_a.shape

(10, 15)

In [115]:
cosine_similarity(vec_b, vec_a) == cosine_similarity(vec_a, vec_b)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [118]:
with open("dictionary.txt", "r") as dict_txt:
	words = [i.strip() for i in dict_txt.readlines()]

In [119]:
words[0]

'a'

In [120]:
new_indices = []
new_words = []

for w in words:
	try:
		new_indices.append(filtered_word2index_2[w])
		new_words.append(w)
	except Exception as e:
		pass

In [121]:
len(new_indices)

76310

In [122]:
new_words[:5]

['aa', 'aah', 'aahed', 'aahing', 'aahs']

In [123]:
new_indices[:5]

[23709, 201333, 277855, 266811, 125567]

In [124]:
with open("filtered_word2index.json", "w") as filt_w2i_json:
	json.dump([i.lower() for i in new_words], filt_w2i_json)

In [129]:
new_np_vec = w.vectors[new_indices]

In [130]:
with open("embed_vec_filtered.npy", "wb") as filtered_npy:
	np.save(filtered_npy, new_np_vec)